In [50]:
from datetime import datetime, timedelta
from dotenv import dotenv_values
from plotting_auxiliary import m_aep_func, figuring
import numpy as np
import os
import pandas as pd
import pickle
import math
import warnings
dir_path = dotenv_values()["DIR_PATH"]
warnings.filterwarnings("ignore")

In [51]:
new_schema = True
if new_schema:
    # date = datetime.strptime("2023-03-02", "%Y-%m-%d")
    date0 = datetime.now() + timedelta(days=-4)
    date = datetime.strptime(str(date0.year) + "-" + str(date0.month) + "-" + str(date0.day), "%Y-%m-%d")
    date_str = date.strftime("%Y_%m_%d")
    rt_file = r"/.cache/outputs/realtime_data.csv"
else:
    date = datetime.strptime("2021-09-27", "%Y-%m-%d")
    date_str = str(date.year) + str(date.month) + str(date.day)
    rt_file = r"/.cache/outputs/realtime_data_old.csv"
mode = 'lab'
bypass_not_read = True
bypass_outband = True
print(date_str)

2023_04_13


In [52]:
if os.path.exists(dir_path + r"/.cache/outputs/for" + date_str + ".pickle"):
    file_to_read = open(dir_path + r'/.cache/outputs/for' + date_str + '.pickle', 'rb')
    print(file_to_read.name + " is loaded.")
    if new_schema:
        grid_inp, v_mage, result_p_pv, result_p_dm, result_q_dm, result_soc, result_price, robust_par = pickle.load(file_to_read)
    else:
        grid_inp = pickle.load(file_to_read)
        v_mage = pickle.load(file_to_read)
        result_p_pv = pickle.load(file_to_read)
        result_p_dm = pickle.load(file_to_read)
        result_q_dm = pickle.load(file_to_read)
        result_soc = pickle.load(file_to_read)
        result_price = pickle.load(file_to_read)
        robust_par = pickle.load(file_to_read)
    file_to_read.close()
else:
    result_p_pv, result_p_dm, result_q_dm, result_soc,  = np.empty((3,144)), np.empty((3,144)), np.empty((3,144)), np.empty((3,1))
    result_p_pv[:], result_p_dm[:], result_q_dm[:], result_soc[:]  = np.NaN, np.NaN, np.NaN, np.NaN
if os.path.exists(dir_path + r"/.cache/outputs/res" + date_str + ".pickle"):
    file_to_read = open(dir_path + r'/.cache/outputs/res' + date_str + '.pickle', 'rb')
    print(file_to_read.name + " is loaded.")
    if new_schema:
        try:
            p_sc, q_sc, rpp_sc, rpn_sc, rqp_sc, rqn_sc, soc_desired, prices_vec, obj = pickle.load(file_to_read)
        except ():
            p_sc, q_sc, rpp_sc, rpn_sc, rqp_sc, rqn_sc, soc_desired, prices_vec = pickle.load(file_to_read)
            obj = None
    else:
        p_sc = pickle.load(file_to_read)
        q_sc= pickle.load(file_to_read)
        rpp_sc = pickle.load(file_to_read)
        rpn_sc = pickle.load(file_to_read)
        rqp_sc = pickle.load(file_to_read)
        rqn_sc = pickle.load(file_to_read)
        soc_desired = pickle.load(file_to_read)
        prices_vec = pickle.load(file_to_read)
        try:
            obj = pickle.load(file_to_read)
        except ():
            obj = None
    file_to_read.close()
else:
    if new_schema:
        p_sc  = np.empty((1,144))
        q_sc= np.empty((1,144))
        rpp_sc = np.empty((1,144))
        rpn_sc = np.empty((1,144))
        rqp_sc = np.empty((1,144))
        rqn_sc= np.empty((1,144))
        soc_desired  = np.empty((1,144))
    else:
        p_sc  = np.empty(144)
        q_sc= np.empty(144)
        rpp_sc = np.empty(144)
        rpn_sc = np.empty(144)
        rqp_sc = np.empty(144)
        rqn_sc= np.empty(144)
        soc_desired  = np.empty(144)
    p_sc[:] = np.NaN
    q_sc[:] = np.NaN
    rpp_sc[:] = np.NaN
    rpn_sc[:] = np.NaN
    rqp_sc[:] = np.NaN
    rqn_sc[:] = np.NaN
    soc_desired[:] = np.NaN
    obj = None
print("DA Objective:", obj)

D:/digriflex/.cache/outputs/for2023_04_13.pickle is loaded.
D:/digriflex/.cache/outputs/res2023_04_13.pickle is loaded.
DA Objective: -817.5485476822553


In [53]:
columns=['0', 'running_time', 'date', 'pv_max_lag', 'irr_lag', 'pdem_lag', 'qdem_lag',
        'pv_max_for', 'irr_for', 'pdem_for', 'qdem_for', 'pv_sim', 'fp_sim', 'fq_sim',
        'pv_real', 'pv_q_real', 'SOC_real', 'fp_real', 'fq_real', 'asked_P', 'asked_Q',
        'Battery_P_sp', 'Battery_Q_sp', 'ABB_P_sp', 'ABB_c_sp']
data_rt = pd.read_csv(dir_path + rt_file, names=columns, header=None)
data_rt['running_time'] = pd.to_datetime(data_rt['running_time'])
data_rt = data_rt[data_rt['running_time'] >= date]
data_rt = data_rt[data_rt['running_time'] < (date + timedelta(days=1))]
x = pd.date_range(pd.Timestamp(date).floor(freq ='D'), periods=144, freq='10min')
date = data_rt['running_time']
pv_max_lag = data_rt['pv_max_lag'].values
irr_lag = data_rt['irr_lag'].values
pdem_lag = data_rt['pdem_lag'].values
qdem_lag = data_rt['qdem_lag'].values
pv_max_for = data_rt['pv_max_for'].values / 1000
irr_for = data_rt['irr_for'].values
pdem_for = data_rt['pdem_for'].values / 10
qdem_for = data_rt['qdem_for'].values / 10
pv_sim = data_rt['pv_sim'].values
fp_sim = data_rt['fp_sim'].values
fp_sim = [-float(i) for i in fp_sim]
fq_sim = data_rt['fq_sim'].values
fq_sim = [-float(i) for i in fq_sim]
pv_real = data_rt['pv_real'].values
pv_real = [float(i) for i in pv_real]
pv_q_real = data_rt['pv_q_real'].values
soc_real = data_rt['SOC_real'].values
fp_real = data_rt['fp_real'].values
fq_real = data_rt['fq_real'].values
asked_p = - data_rt['asked_P'].values
asked_q = data_rt['asked_Q'].values
bat_p_sp = data_rt['Battery_P_sp'].values
bat_q_sp = data_rt['Battery_Q_sp'].values

In [54]:
date_soc = date
if bypass_not_read:
    tmp1 = date.tolist()
    tmp2 = soc_real.tolist()
    tmp3 = []
    for x1 in tmp1:
        xx = x1.hour * 6 + math.floor(x1.minute / 10)
        tmp3.append(soc_desired[xx])
    index = [i for i in range(len(tmp2)) if abs(tmp2[i] - tmp3[i]) > 0.15]
    date_soc = [tmp1[i] for i in index]
    soc_real = [tmp2[i] for i in index]

if bypass_outband:
    tmp1 = date_soc
    tmp2 = soc_real
    tmp3 = []
    for x1 in tmp1:
        xx = x1.hour * 6 + math.floor(x1.minute / 10)
        tmp3.append(soc_desired[xx])
    index = [i for i in range(len(tmp2)) if abs(tmp2[i] - tmp3[i]) < 100]
    date_soc = [tmp1[i] for i in index]
    soc_real = [tmp2[i] for i in index]
date_p, date_q = date, date
if bypass_outband:
    tmp1 = date.tolist()
    tmp2 = fp_real.tolist()
    tmp2_2 = asked_p.tolist()
    tmp3, tmp4, tmp5 = [], [], []
    for x1 in tmp1:
        xx = x1.hour * 6 + math.floor(x1.minute / 10)
        tmp3.append(p_sc[xx])
        tmp4.append(rpp_sc[xx])
        tmp5.append(rpn_sc[xx])
    index = [i for i in range(len(tmp2)) if (tmp2[i] - tmp3[i] - tmp4[i]) < 5 and (tmp2[i] - tmp3[i] + tmp5[i]) > -5]
    date_p = [tmp1[i] for i in index]
    fp_real = [tmp2[i] for i in index]
    asked_p = [tmp2_2[i] for i in index]
if bypass_outband:
    tmp1 = date.tolist()
    tmp2 = fq_real.tolist()
    tmp2_2 = asked_q.tolist()
    tmp3, tmp4, tmp5 = [], [], []
    for x1 in tmp1:
        xx = x1.hour * 6 + math.floor(x1.minute / 10)
        tmp3.append(q_sc[xx])
        tmp4.append(rqp_sc[xx])
        tmp5.append(rqn_sc[xx])
    index = [i for i in range(len(tmp2)) if (tmp2[i] - tmp3[i] - tmp4[i]) < 5 and (tmp2[i] - tmp3[i] + tmp5[i]) > -5]
    date_q = [tmp1[i] for i in index]
    fq_real = [tmp2[i] for i in index]
    asked_q = [tmp2_2[i] for i in index]

In [55]:
pdem_for[pdem_for==0] = pdem_lag[pdem_for==0]
qdem_for[qdem_for==0] = qdem_lag[qdem_for==0]

In [56]:
if mode == 'sim':
    # Demanded active power
    m_aep = m_aep_func(result_p_dm[0, :], pdem_lag, date, False)
    print('DA Forecast, MAE (kW):', m_aep)
    m_aep = m_aep_func(pdem_for, pdem_lag, date, True)
    print('RT Forecast, MAE (kW):', m_aep)
    figuring(x, result_p_dm, 'DP', date, [], pdem_for, pdem_lag)
    # Demanded reactive power
    m_aep = m_aep_func(result_q_dm[0, :], qdem_lag, date, False)
    print('DA Forecast, MAE (kVar):', m_aep)
    m_aep = m_aep_func(qdem_for, qdem_lag, date, True)
    print('RT Forecast, MAE (kVar):', m_aep)
    figuring(x, result_q_dm, 'DQ', date, [], qdem_for, qdem_lag)
    # PV production
    m_aep = m_aep_func(result_p_pv[0, :], pv_max_lag, date, False)
    print('DA Forecast, MAE (kW):', m_aep)
    m_aep = m_aep_func(pv_max_for, pv_max_lag, date, True)
    print('RT Forecast, MAE (kW):', m_aep)
    figuring(x, result_p_pv, 'PV', date, pv_max_lag, pv_max_for, pv_sim)
    # ConnectionPoint P
    figuring(x, np.array([p_sc, rpp_sc, rpn_sc]), 'FP', date_p, [], asked_p, fp_sim)
    # ConnectionPoint Q
    figuring(x, np.array([q_sc, rqp_sc, rqn_sc]), 'FQ', date_q, [], asked_q, fq_sim)
elif mode == 'lab':
    # Demanded active power
    m_aep = m_aep_func(result_p_dm[0, :], pdem_lag, date, False)
    print('DA Forecast, MAE (kW):', m_aep)
    m_aep = m_aep_func(pdem_for, pdem_lag, date, True)
    print('RT Forecast, MAE (kW):', m_aep)
    figuring(x, result_p_dm, 'DP', date, [], pdem_for, pdem_lag)
    # Demanded reactive power
    m_aep = m_aep_func(result_q_dm[0, :], qdem_lag, date, False)
    print('DA Forecast, MAE (kVar):', m_aep)
    m_aep = m_aep_func(qdem_for, qdem_lag, date, True)
    print('RT Forecast, MAE (kVar):', m_aep)
    figuring(x, result_q_dm, 'DQ', date, [], qdem_for, qdem_lag)
    # PV production
    m_aep = m_aep_func(result_p_pv[0, :], pv_max_lag, date, False)
    print('DA Forecast, MAE (kW):', m_aep)
    m_aep = m_aep_func(pv_max_for, pv_max_lag, date, True)
    print('RT Forecast, MAE (kW):', m_aep)
    figuring(x, result_p_pv, 'PV', date, pv_max_lag, pv_max_for, pv_real)
    # ConnectionPoint P
    figuring(x, np.array([p_sc, rpp_sc, rpn_sc]), 'FP', date_p, [], asked_p, fp_real)
    # ConnectionPoint Q
    figuring(x, np.array([q_sc, rqp_sc, rqn_sc]), 'FQ', date_q, [], asked_q, fq_real)
# SoC
soc_pos = [result_soc[1] * result_soc[0]] * 144
soc_pos = [max([soc_pos[ii], 90 - soc_desired[ii]]) for ii in range(144)]
soc_neg = [result_soc[2] * result_soc[0]] * 144
soc_neg = [max([soc_neg[ii], soc_desired[ii] - 10]) for ii in range(144)]
figuring(x, np.array([soc_desired, soc_pos, soc_neg]), 'SOC', date_soc, [], [], soc_real)
figuring(x, np.array([p_sc, rpp_sc, rpn_sc]), 'p_bat_setpoint', date_p, [], bat_p_sp,[])
figuring(x, np.array([q_sc, rqp_sc, rqn_sc]), 'q_bat_setpoint', date_q, [], bat_q_sp,[])


DA Forecast, MAE (kW): 0.9400828049007732
RT Forecast, MAE (kW): 0.5120210894308944


DA Forecast, MAE (kVar): 0.15329518485044
RT Forecast, MAE (kVar): 0.034567906504065035


DA Forecast, MAE (kW): 1.5899640758568132
RT Forecast, MAE (kW): 0.674959683292683
